In [1]:
import numpy as np
import pandas as pd
from donut import complete_timestamp, standardize_kpi, Donut, DonutTrainer, DonutPredictor
from tensorflow import keras as K
from tfsnippet.modules import Sequential
from tfsnippet.utils import get_variables_as_dict, VariableSaver
import tensorflow.compat.v1 as tf
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
tf.disable_v2_behavior()
 
# path to the dataset
file_csv = "cpu4.csv"
 
# Read the raw data.
data = pd.read_csv(file_csv)
timestamp = data["timestamp"]
values = data["value"]
labels = data["label"]
dataset_name = file_csv.split('.')[0]
print("Timestamps: {}".format(timestamp.shape[0]))

ModuleNotFoundError: No module named 'pandas'

In [2]:
pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 11.3 MB 553 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 24.8 MB 623 kB/s eta 0:00:01
     |████████████████████████████████| 306 kB 657 kB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 651 kB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=806fb46565046659de59a3fb0900b1867574dad711f9a9fff2d8b28055b3eb86
  Stored in directory: /root/.cache/pip/wheels/23/51/c2/5159d7bbfa5e6ba511213e0317a65331f1174d1c155af26def
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from donut import complete_timestamp, standardize_kpi, Donut, DonutTrainer, DonutPredictor
from tensorflow import keras as K
from tfsnippet.modules import Sequential
from tfsnippet.utils import get_variables_as_dict, VariableSaver
import tensorflow.compat.v1 as tf
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
tf.disable_v2_behavior()
 
# path to the dataset
file_csv = "cpu4.csv"
 
# Read the raw data.
data = pd.read_csv(file_csv)
timestamp = data["timestamp"]
values = data["value"]
labels = data["label"]
dataset_name = file_csv.split('.')[0]
print("Timestamps: {}".format(timestamp.shape[0]))





Instructions for updating:
non-resource variables are not supported in the long term
Timestamps: 17568


In [5]:
# Complete the timestamp filling missing points with zeros, and obtain the missing point indicators.
# 该complete_timestamp函数用 0 填充缺失点，并返回一个数组，指示它们在增强数据集中的位置。
timestamp, missing, (values, labels) = complete_timestamp(timestamp, (values, labels))
print("Missing points: {}".format(np.sum(missing == 1)))
print("Labeled anomalies: {}".format(np.sum(labels == 1)))

Missing points: 0
Labeled anomalies: 209


In [6]:
# Split the training and testing data.
# 分离为训练集和测试集
test_portion = 0.3
test_n = int(len(values) * test_portion)
train_values, test_values = values[:-test_n], values[-test_n:]
train_labels, test_labels = labels[:-test_n], labels[-test_n:]
train_missing, test_missing = missing[:-test_n], missing[-test_n:]
print("Rows in test set: {}".format(test_values.shape[0]))
print("Anomalies in test set: {}".format(np.sum(test_labels == 1)))

Rows in test set: 5270
Anomalies in test set: 14


In [7]:
# Standardize the training and testing data, anomaly points or missing points are excluded
# 对训练集和测试集进行标准化，从均值和标准差的计算中排除缺失点或异常。
train_values, mean, std = standardize_kpi(
    train_values, excludes=np.logical_or(train_labels, train_missing))
test_values, _, _ = standardize_kpi(test_values, mean=mean, std=std)
print("Train values mean: {}".format(mean))
print("Train values std: {}".format(std))

Train values mean: 0.008981638588011265
Train values std: 0.9900397062301636


In [8]:
sliding_window = 120
 
# define the model inside the 'model_vs' scope
# 在“model_vs”的范围内定义构建它的模型
with tf.variable_scope('model') as model_vs:
    model = Donut(
        h_for_p_x=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        h_for_q_z=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        x_dims=sliding_window,
        z_dims=5,
    )
# use DonutTrainer class to train the model
# 该类DonutTrainer用于训练模型
trainer = DonutTrainer(model=model, model_vs=model_vs, max_epoch=30)
# use DonutPredictor class to make predictions
# 同时通过该类进行预测DonutPredictor
predictor = DonutPredictor(model)






Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [9]:
# 将对其进行训练并创建一个文件来保存它，以便我们将来可以重用相同的模型来进行新的预测，从而避免再次保留它。
save_dir = "model1/" + dataset_name + "/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
saved = True
if len(os.listdir(save_dir)) == 0:
    saved = False
 
if saved is False:
    with tf.Session().as_default():
        # train the model
        trainer.fit(train_values, train_labels, train_missing, mean, std)
        # save variables to 'save_dir' directory
        var_dict = get_variables_as_dict(model_vs)
        saver = VariableSaver(var_dict, save_dir)
        saver.save()
        saved = True

2022-05-21 18:38:44.454036: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-05-21 18:38:44.462682: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
2022-05-21 18:38:44.466292: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5618fee4b490 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-21 18:38:44.466312: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



Trainable Parameters                     (58,150 in total)
----------------------------------------------------------
donut/p_x_given_z/x_mean/bias           (120,)         120
donut/p_x_given_z/x_mean/kernel         (100, 120)  12,000
donut/p_x_given_z/x_std/bias            (120,)         120
donut/p_x_given_z/x_std/kernel          (100, 120)  12,000
donut/q_z_given_x/z_mean/bias           (5,)             5
donut/q_z_given_x/z_mean/kernel         (100, 5)       500
donut/q_z_given_x/z_std/bias            (5,)             5
donut/q_z_given_x/z_std/kernel          (100, 5)       500
sequential/forward/_0/dense/bias        (100,)         100
sequential/forward/_0/dense/kernel      (5, 100)       500
sequential/forward/_1/dense_1/bias      (100,)         100
sequential/forward/_1/dense_1/kernel    (100, 100)  10,000
sequential_1/forward/_0/dense_2/bias    (100,)         100
sequential_1/forward/_0/dense_2/kernel  (120, 100)  12,000
sequential_1/forward/_1/dense_3/bias    (100,)         

In [10]:
# 模型经过训练 就可以在预测阶段结束代码
if saved:
    with tf.Session().as_default():
        # restore variables from 'save_dir'
        # 在"save_dir"中储存变量
        saver = VariableSaver(get_variables_as_dict(model_vs), save_dir)
        saver.restore()
        # make predictions 做出检测
        test_score = predictor.get_score(test_values, test_missing)
        print("Number of predictions: {}".format(test_score.shape[0]))
        # try different thresholds
        # 使用不同的阈值
        best_threshold = 0
        best_f1 = 0
        best_predictions = []
        thresholds = np.arange(5, 50, 0.2)
        for t in thresholds:
            threshold = t  # can be changed to better fit the training data
            anomaly_predictions = []
            for l in test_score:
                if abs(l) > threshold:
                    anomaly_predictions.append(1)
                else:
                    anomaly_predictions.append(0)
            # strategy to compute modified metrics
            # 计算修改后的指标的策略
            
            for i in range(sliding_window-1, len(anomaly_predictions)):
                if anomaly_predictions[i-sliding_window+1] == 1 and test_labels[i] == 1:  # true positive
                    j = i-1
                    while j >= sliding_window-1 and test_labels[j] == 1\
                            and anomaly_predictions[j-sliding_window+1] == 0:
                        anomaly_predictions[j-sliding_window+1] = 1
                        j -= 1
                    j = i+1
                    while j < len(anomaly_predictions) and test_labels[j] == 1\
                            and anomaly_predictions[j-sliding_window+1] == 0:
                        anomaly_predictions[j-sliding_window+1] = 1
                        j += 1
            f1 = f1_score(test_labels[sliding_window-1:], anomaly_predictions, average='binary')
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
                best_predictions = anomaly_predictions
 
        anomaly_predictions = np.array(best_predictions)
        print("--  final results --")
        print("Best anomaly threshold {}".format(best_threshold))
        print("Anomalies found: {}/{}".format(np.sum(anomaly_predictions == 1), np.sum(test_labels == 1)))
        prfs = precision_recall_fscore_support(test_labels[sliding_window-1:], anomaly_predictions)
        print("--- anomaly rows ---")
        print("precision: {:.3f}".format(prfs[0][1]))
        print("recall: {:.3f}".format(prfs[1][1]))
        print("fscore: {:.3f}".format(prfs[2][1]))

INFO:tensorflow:Restoring parameters from /root/autodl-tmp/model1/cpu4/variables.dat

Number of predictions: 5151
--  final results --
Best anomaly threshold 6.800000000000002
Anomalies found: 10/14
--- anomaly rows ---
precision: 1.000
recall: 0.714
fscore: 0.833
